In this notebook, I will be pulling imagery from the year before a fire in the Great Smoky Mountains and after the fire. We will be comparing NDVI, NBR, and MSMI and pulling species occurrence records from before and after the fire and see how it overlaps with occurrence records.

#### Declare ROI

In [8]:
import ee
import geemap

# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize(project="ee-krle4401")

site_center = ee.Geometry.Point([-83.5, 35.7])

ModuleNotFoundError: No module named 'StringIO'

#### Pull NEON Imagery from GEE

In [ ]:
# Load AOP Image Collection
sdr_col = ee.ImageCollection('projects/neon-prod-earthengine/assets/HSI_REFL/001')

# Retrieve all available image IDs
image_ids = sdr_col.aggregate_array("system:index").getInfo()
print("🔍 Available AOP Image IDs:", image_ids)

# **Function to Filter Images by Multiple Years & Domains**
def filter_aop_images(years, domains):
    """
    Filters the AOP image collection based on a list of years and domains.
    
    Parameters:
        years (list): List of years (e.g., ["2016", "2017"]).
        domains (list): List of 4-letter site codes (e.g., ["GRSM", "HARV"]).
    
    Returns:
        Dictionary with (year, domain) as key and list of matching image IDs.
    """
    filtered_results = {}

    for year in years:
        for domain in domains:
            matching_ids = [img_id for img_id in image_ids if year in img_id and domain in img_id]
            if matching_ids:
                filtered_results[(year, domain)] = matching_ids

    return filtered_results

# **User Inputs**
years_input = ["2016", "2017"]  # Add multiple years here
domains_input = ["GRSM"]  # Add multiple domains here

# Get filtered images
filtered_images = filter_aop_images(years_input, domains_input)

if not filtered_images:
    print(f"No AOP images found for the selected years and domains.")
else:
    print(f"Found AOP images: {filtered_images}")

# NDVI Visualization Parameters
ndvi_vis_params = {
    'min': -1, 'max': 1,
    'palette': ['blue', 'white', 'green']
}

# Create a geemap Map instance
Map = geemap.Map()
Map.centerObject(site_center, 10)

# Loop through filtered images and compute NDVI
for (year, domain), image_ids in filtered_images.items():
    for image_id in image_ids:
        # Retrieve the image by its system:index
        aop_image = sdr_col.filter(ee.Filter.eq("system:index", image_id)).first()

        # Function to Compute NDVI
        def addAOPBands(image):
            ndvi = image.normalizedDifference(['B097', 'B055']).rename('NDVI')
            return image.addBands(ndvi).set({'Sensor': 'AOP'})

        # Compute NDVI
        if aop_image:
            aop_ndvi = addAOPBands(aop_image).select('NDVI')
            print(f"NDVI computed for AOP image: {image_id}")

            # Add layer with proper naming
            Map.addLayer(aop_ndvi, ndvi_vis_params, f'AOP NDVI ({image_id})')

# Save the final interactive HTML map with all selected images
html_filename = f"AOP_NDVI_{'_'.join(years_input)}_{'_'.join(domains_input)}.html"
Map.to_html(filename=html_filename)

print(f"NDVI visualization saved: {html_filename}")


/Users/kitlewers/NEON_GBIF_Burn_Scar/env/lib/python3.13/site-packages/ee/main.py:150: SyntaxWarning: invalid escape sequence '\d'
  lines = filter(lambda x: re.match("^\d+ bytes", x), data.splitlines())


ModuleNotFoundError: No module named 'StringIO'

#### Pull Records from GBIF